# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

import nltk
nltk.download(['punkt', 'wordnet'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score 
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import FunctionTransformer
from sklearn.multioutput import MultiOutputClassifier

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///figure_8_messages.db')
df = pd.read_sql("SELECT * FROM figure_8_messages", engine)
# exclude id and original columns for X data
# X = df[["message", "genre"]]
X = df[["message", "genre"]]
y = df.iloc[:,4:]
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = [lemmatizer.lemmatize(tok).lower().strip() for tok in tokens]

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [124]:
def get_message_col(X):
    """
    Return only message column
    """
    return X["message"]

def get_genre_col(X):
    """
    Return only genre column
    """
    return X["genre"]

def get_dummies(X):
    """
    one-hot encode the categorical variables
    """
    return pd.get_dummies(X)

text_pipeline = Pipeline([
    ('get_text', FunctionTransformer(get_message_col, validate=False)),
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer())
])

genre_pipeline = Pipeline([
    ('get_genre', FunctionTransformer(get_genre_col, validate=False)),
    ('get_dummies', FunctionTransformer(get_dummies, validate=False))
])

features = FeatureUnion([
    ('text_pipeline', text_pipeline),
    ('genre_pipeline', genre_pipeline)
])

# BaggingClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
# GaussianNB, SVC

model = RandomForestClassifier(n_estimators=5,
                              min_samples_split=6)

pipeline = Pipeline([
    ('features', features),
    ('clf', MultiOutputClassifier(model, n_jobs=-1))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [125]:
# perform train test split
# X_train, X_test, y_train, y_test = train_test_split(X, y)

In [126]:
%%time
# train classifier
pipeline.fit(X_train, y_train)

# predict on test data
y_pred = pipeline.predict(X_test)

CPU times: user 40.1 s, sys: 7.98 ms, total: 40.1 s
Wall time: 40.2 s


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [127]:
recall_list = list()
accuracy_list = list()
precision_list = list()
for idx in range(y_test.columns.shape[0]):
    test = y_test[y_test.columns[idx]].values
    pred = y_pred[:,idx]
    
    accu = accuracy_score(test, pred)
    accuracy_list.append(accu)
    prec = precision_score(test, pred)
    precision_list.append(prec)
    reca = recall_score(test, pred)
    recall_list.append(reca)
    print(f"{y_test.columns[idx]} | accuracy: {accu:.2f} | precision: {prec:.2f} | recall: {reca:.2f}")

related | accuracy: 0.78 | precision: 0.81 | recall: 0.93
request | accuracy: 0.88 | precision: 0.75 | recall: 0.46
offer | accuracy: 0.99 | precision: 0.00 | recall: 0.00
aid_related | accuracy: 0.72 | precision: 0.68 | recall: 0.61
medical_help | accuracy: 0.92 | precision: 0.47 | recall: 0.11
medical_products | accuracy: 0.95 | precision: 0.47 | recall: 0.07
search_and_rescue | accuracy: 0.97 | precision: 0.25 | recall: 0.01
security | accuracy: 0.98 | precision: 0.25 | recall: 0.02
military | accuracy: 0.97 | precision: 0.50 | recall: 0.09
child_alone | accuracy: 1.00 | precision: 0.00 | recall: 0.00
water | accuracy: 0.95 | precision: 0.81 | recall: 0.26
food | accuracy: 0.92 | precision: 0.75 | recall: 0.36
shelter | accuracy: 0.93 | precision: 0.78 | recall: 0.30
clothing | accuracy: 0.98 | precision: 0.53 | recall: 0.15
money | accuracy: 0.98 | precision: 0.42 | recall: 0.06
missing_people | accuracy: 0.99 | precision: 0.25 | recall: 0.01
refugees | accuracy: 0.97 | precision: 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [128]:
print(f'avg. recall {np.array(recall_list).mean():.3f}')
print(f'avg. precision {np.array(precision_list).mean():.3f}')
print(f'avg. accuracy {np.array(accuracy_list).mean():.3f}')

avg. recall 0.183
avg. precision 0.489
avg. accuracy 0.940


In [129]:
for idx in range(y_test.columns.shape[0]):
    test = y_test[y_test.columns[idx]].values
    pred = y_pred[:,idx]
    print(f"{y_test.columns[idx]}")
    print(classification_report(test, pred))

related
             precision    recall  f1-score   support

          0       0.60      0.31      0.41      1576
          1       0.81      0.93      0.87      4978

avg / total       0.76      0.78      0.76      6554

request
             precision    recall  f1-score   support

          0       0.90      0.97      0.93      5425
          1       0.75      0.46      0.57      1129

avg / total       0.87      0.88      0.87      6554

offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6516
          1       0.00      0.00      0.00        38

avg / total       0.99      0.99      0.99      6554

aid_related
             precision    recall  f1-score   support

          0       0.75      0.80      0.77      3854
          1       0.68      0.61      0.64      2700

avg / total       0.72      0.72      0.72      6554

medical_help
             precision    recall  f1-score   support

          0       0.93      0.99      0

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


             precision    recall  f1-score   support

          0       0.99      1.00      0.99      6480
          1       0.00      0.00      0.00        74

avg / total       0.98      0.99      0.98      6554

other_infrastructure
             precision    recall  f1-score   support

          0       0.96      1.00      0.98      6271
          1       0.26      0.02      0.03       283

avg / total       0.93      0.96      0.94      6554

weather_related
             precision    recall  f1-score   support

          0       0.85      0.93      0.88      4728
          1       0.75      0.57      0.64      1826

avg / total       0.82      0.83      0.82      6554

floods
             precision    recall  f1-score   support

          0       0.94      0.99      0.97      5996
          1       0.82      0.37      0.51       558

avg / total       0.93      0.94      0.93      6554

storm
             precision    recall  f1-score   support

          0       0.94      0.98    

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
pipeline.get_params().keys()

In [ ]:
pipeline.get_params()["clf__estimator__min_samples_split"]

In [ ]:
%%time
parameters = {
    'clf__estimator__n_estimators': [5, 10, 50, 100],
    "clf__estimator__min_samples_split": [2, 4, 10]
    }

cv = GridSearchCV(pipeline, param_grid=parameters)

cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
for idx in range(y_test.columns.shape[0]):
    test = y_test[y_test.columns[idx]].values
    pred = y_pred[:,idx]
    print(f"{y_test.columns[idx]}")
    print(f"accuracy: {accuracy_score(test, pred)}")
    print(f"precision: {accuracy_score(test, pred)}")
    print(f"recall: {accuracy_score(test, pred)}")

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [130]:
from sklearn.externals import joblib
joblib.dump(pipeline, 'my_model.pkl')

['my_model.pkl']

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.